In [1]:
import numpy as np
import pandas as pd
storage_options = {'User-Agent': 'Mozilla/5.0'}
url = "https://www.fraserinstitute.org/sites/default/files/economic-freedom-of-north-america-2022-for-website-allgovs.xlsx"
# GDP = pd.read_csv("StateGDP.csv")
efw = pd.read_excel(url, storage_options=storage_options, sheet_name=None)
codes = pd.read_csv("taxitemcodesanddescriptions.csv", header = [0], index_col = [0])
# list(GDP[["LineCode","Description"]].set_index(["LineCode", "Description"]).index.unique())

In [2]:
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 
    'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA','Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

fips_dict = {
 '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR',
 '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE',
 '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID',
 '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS',
 '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
 '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS',
 '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV',
 '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY',
 '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK',
 '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC',
 '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT',
 '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV',
 '55': 'WI', '56': 'WY'}
# '60': 'AS', '66': 'GU',
#  '69': 'MP', '72': 'PR', '78': 'VI'}
state_df = pd.DataFrame(state_dict, index = ["Abbr"]).T
fips_df = pd.DataFrame(fips_dict, index = ["State"]).T
state_names, state_abbrev = state_df.index, list(state_df["Abbr"].values)
state_df.T# state_abbrev

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
Abbr,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY


In [3]:
#FRED.py
import pandas as pd
import pandas_datareader.data as web
import datetime

def gather_data(data_codes, 
                start, 
                end = datetime.datetime.today(), 
                freq = "M"):
    i = 0
    # dct.items() calls key and value that key points to
    for key, val in data_codes.items():
        if i == 0:
            # Create dataframe for first variable, then rename column
            df = web.DataReader(
                val, "fred", start, end).resample(freq).mean()
            df.rename(columns = {val:key}, inplace = True) 
            # setting i to None will cause the next block of code to execute,
            # placing data within df instead of creating a new dataframe for
            # each variable
            i = None
        else:
            # If dataframe already exists, add new column
            df[key] = web.DataReader(val, "fred", start, end).resample(freq).mean()

    return df

In [4]:
data_codes  = {state: f"{state}POP" for state in state_dict.values()}
state_gdp_codes = {state: f"{state}NGSP" for state in state_dict.values()}

# Select start and end dates
start = datetime.datetime(1985, 1, 1)
end = datetime.datetime.today()

# freq refers to data frequency. Choose "D", "W", "M", "Q", "A"
# a number may also be place in front of a letter. "2D" indicates
#       alternating days
fred_data = {"Population": gather_data(data_codes = data_codes, start = start, end = end, freq = "A"),
             "GDP": gather_data(data_codes = state_gdp_codes, start = start, end = end, freq = "A")}

In [5]:
panels = {}
for key, fred_df in fred_data.items():
    print(key)
    panels[key] = fred_df.reset_index()
    panels[key]["DATE"] = panels[key]["DATE"].astype(str).str[:4].astype(int)
    panels[key] = panels[key].melt(
        id_vars = ["DATE"], 
        var_name = "State", 
        value_name = key).set_index(["State", "DATE"]).round(2)

    
panels[key]


Population
GDP


GDP
State DATE          
AL    1997  105381.4
      1998  110838.8
      1999  116514.3
      2000  120522.6
      2001  123534.5
...              ...
WY    2019   39971.4
      2020   36675.5
      2021   42176.2
      2022   49080.6
      2023   50171.6

[1350 rows x 1 columns]

In [6]:
years = [1985,
 1990,
 1995,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020]
for name, efwdata in list(efw.items())[1:]:
        cols = list(efwdata.keys())
        col0 = cols[0]
        efwdata = efwdata.dropna(subset = cols[-1]).rename(columns = {col0:"State"})
        panel = efwdata[efwdata["State"].isin(state_names)]
        for i in range(len(years)):
                try:
                        panel = panel[["State"] + years[i:]]
                        break
                except:
                        continue
        panel["State"] = panel["State"].apply(lambda x:state_dict[x])
        panel = panel.melt(id_vars=["State"], 
                var_name="Year", 
                value_name=name).set_index(["State", "Year"]).round(2)
        panels[name] = panel.copy()
panels["EFNA"] = panels['Overall-scores(allgovs)'].rename(columns = {'Overall-scores(allgovs)':"EFNA"}).copy()
panel_keys = list(panels.keys())
for k in panel_keys:
        panels[k].rename(columns = {k: k.replace("-allg","")}, inplace = True)
        if k != k.replace("-allg",""):
                panels[k.replace("-allg","")] = panels[k].copy()
                del panels[k]
panels.keys()

dict_keys(['Population', 'GDP', 'Overall-scores(allgovs)', 'Overall-ranks(allgovs)', 'EFNA', '1A ', '1B', '1C', '1D', '2A', '2B', '2Bi', '2Bii', '2C', '2D', '3A', '3Ai', '3Aii', '3Aiii', '3Aiv', '3B', '3C', '4', '5', '6'])

In [7]:
expand_digits = {"Population": 3, "GDP": 6}#, "EFW":0}
for key in efw.keys():
    expand_digits[key] = 0
panel = panels["Population"].copy()
for key, df in panels.items():
    panel[key] = df[key].mul(10 ** expand_digits[key])
panel["GDP Per Capita"] = panel["GDP"].div(panel["Population"])
for key in ("GDP", "Population", "GDP Per Capita"):
    panel[f"Log {key}"] = np.log(panel[key])
    panel[f"{key} % Growth"] = panel[f"Log {key}"].groupby("State").diff()
panel.loc["ND"]

KeyError: 'EFNA'

In [ ]:
df

In [ ]:
# Maybe labels from 97 keys were entered by an intern who was filling in for the usual data person who was on leave,
fix_97_keys = pd.DataFrame({'VI':"VA", "AR":"AZ", 
                            'AR    .1':"AR", "LS":"LA",
                            "GE":"GA"}, 
                           index = ["Correction"]).T
fix_97_keys

In [ ]:
# urls = ["https://www2.census.gov/govs/statetax/" +str(i)[-2:] + "staxcd.txt" for i in range(92,122)]
tax_dfs = {}
for year in range(1992,2022):
    key = str(year)[-2:] 
    try:    
        url = "https://www2.census.gov/govs/statetax/" +key + "staxcd.txt"
        tax_dfs[year] = pd.read_csv(url, index_col = [0])
        tax_dfs[year].rename(columns = {col:col.strip().replace("-State","") for col in tax_dfs[year].columns}, inplace = True)
        tax_dfs[year].rename(columns = {name:state_df.loc[name].values[0] for name in state_names}, inplace = True)
        if key == "97":
            tax_dfs[year].rename(columns = {
                wrong_key:fix_97_keys.loc[wrong_key].values[0] for wrong_key in fix_97_keys.index}, 
                                inplace = True)
        # adjust units from 1000s to millions
        tax_dfs[year] = tax_dfs[year][state_abbrev].div(1000)
#         tax_dfs[key].rename(columns = {col:col.strip() for col in tax_dfs[key].columns}, inplace = True)
    except:
        print("failed to download " + url)
tax_dfs.keys()

In [ ]:
code_dfs = {}
for code in codes.index:
    code_dfs[code] = {}
    for year in tax_dfs.keys():
        code_dfs[code][year] = tax_dfs[year].loc[code] 
    code_dfs[code] = pd.DataFrame(code_dfs[code])

In [ ]:
code_dfs["TotalTax"] = {}
for key, df in tax_dfs.items():
    print(df.index)
    code_dfs["TotalTax"][key] = df.iloc[1:].sum()
# code_dfs["TotalTax"]

In [ ]:
tax_names = list(code_dfs.keys())
for code, df in code_dfs.items():
    code_dfs[code] = pd.DataFrame(df)
    code_dfs[code].index.name = "State"
    code_dfs[code] = code_dfs[code].reset_index().melt(id_vars=["State"], 
        var_name="Year", 
        value_name=code).set_index(["State", "Year"])
    #     panel.loc[panel.index.isin(code_dfs[code].index),code] = code_dfs[code]
# panel = panel[code].copy()
for code, df in code_dfs.items():
    panel[code] = code_dfs[code]
    
#     panel[code + "/GDP"] = panel[code].div(panel["GDP"]).mul(100)

In [ ]:
panel["TotalTax/GDP"] = panel["TotalTax"].div(panel["GDP"])
panel.dropna()

In [ ]:
panel.reset_index().rename(columns = {"DATE": "Year"}).to_csv("EFNAGDPTaxes.csv")

In [ ]:
import matplotlib.pyplot as plt
no_prop_tax = panel[panel["T01"]==0]
prop_tax = panel[panel["T01"]!=0]

for key in ("GDP Per Capita","GDP", "Population"):
    fig, ax = plt.subplots(figsize = (20,10))
    key = key + " % Growth"
    colors = ["C0", "C3"]
    for i, df in enumerate((no_prop_tax, prop_tax)):
        color = colors[i]
        df.plot.scatter(x = "TotalTax/GDP", y = key, color = color, ax = ax)
        
        
        mean_key = df.mean()[key]
        se_key = df.sem()[key]
        mean_tax_gdp = df.mean()["TotalTax/GDP"]
        se_tax_gdp = df.sem()["TotalTax/GDP"]
        
        
        ax.axhline(mean_key, color = color)
        ax.axhline(mean_key + se_key * 2, color = color, ls = "--")
        ax.axhline(mean_key - se_key * 2, color = color, ls = "--")
        
        ax.axvline(mean_tax_gdp, color = color)
        ax.axvline(mean_tax_gdp + se_tax_gdp * 2, color = color, ls = "--")        
        ax.axvline(mean_tax_gdp - se_tax_gdp * 2, color = color, ls = "--")
        

In [ ]:
pct_gdp_panel = panel.apply(lambda x: x / panel["GDP"])
pct_gdp_panel.dropna()

In [ ]:
# fig, ax = plt.subplots(figsize = (20,10))
# plot_df[tax_names[:-1]].plot.area(
#     legend = True, 
#     ax = ax)
